In [1]:
from collections import Counter
from string import punctuation
import spacy
import numpy as np
import requests
from tqdm import tqdm
from collections import defaultdict
from pprint import pprint
import pandas as pd
from SimilarityFinder import SimilarityFinder
from heapq import heappop, heappush, heapify


c:\Users\shama\anaconda3\envs\flipkart-grid\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = {
    "occasions": ["wedding", "party", "interview", "birthday", "promotion", "anniversary", "trek", "trip"],
    "clothing": ["dress", "shirt", "jacket", "skirt", "jeans", "top", "coord"],
    "footwear": ["shoes", "boots", "sandals", "sneakers", "heels"],
    "accessories": ["bag", "wallet", "watch", "sunglasses", "earrings", "necklaces"],
    "seasons": ["summer", "monsoons", "winter", "spring", "rainy", "cold", "hot"],
    "adjectives": ["pair", "red", "long", "short", "black", "sleeveless", "two"]
}
imp_words = ["party", "interview", "birthday", "dress", "shirt", "jacket", "co-ord", "skirt", "jeans", "top",
                "shoes", "boots", "sandals", "sneakers", "bag", "wallet", "watch", "sunglasses", "summer",
                "monsoons", "winter", "spring"]

categories = ["accessories", "clothing", "footwear", "occasions", "seasons"]



In [15]:
activated = spacy.prefer_gpu()
nlp = spacy.load("en_core_web_lg")
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] 
    doc = nlp(text.lower()) 
    for token in doc:
        # print(token, token.pos_)
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation) and token.text not in imp_words:
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)
    return result
new_text = """
Give me an outfit for this wedding I have to attend next month. I also want a pair of earrings to go with the outfit
"""
output = set(get_hotwords(new_text))
most_common_list = Counter(output).most_common(10)
for item in output:
  print(item)

earrings
outfit
wedding
pair
month


In [16]:
nlp = spacy.blank("en")
nlp.add_pipe(
    "text_categorizer",
    config={
        "data": data,
        "model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        "multi_label": True,
        "device": "gpu"
    }
)
results = {}

for item in tqdm(output):
    doc = nlp(item)
    if max(zip(doc._.cats.values(), doc._.cats.keys()))[1] not in results:
        results[max(zip(doc._.cats.values(), doc._.cats.keys()))[1]] = [item]
    else:
        results[max(zip(doc._.cats.values(), doc._.cats.keys()))[1]].append(item)
    print(item, doc._.cats)

100%|██████████| 5/5 [00:00<00:00, 52.65it/s]

earrings {'accessories': 0.9263005651977559, 'adjectives': 0.1924994779969537, 'clothing': 0.140393548730751, 'footwear': 0.006394503009886536, 'occasions': 0.014464147136777607, 'seasons': 0.00626514756374165}
outfit {'accessories': 0.11027875187822017, 'adjectives': 0.10776789782065695, 'clothing': 0.6524800098756908, 'footwear': 0.10837575587780442, 'occasions': 0.04945828746461525, 'seasons': 0.018292284853725155}
wedding {'accessories': 0.053406147895035685, 'adjectives': 0.15307097217352728, 'clothing': 0.1400920963851825, 'footwear': 0.012116931953655704, 'occasions': 0.9559288755274307, 'seasons': 0.015212197527639566}
pair {'accessories': 0.09344894055163748, 'adjectives': 0.61610841120685, 'clothing': 0.08364111915610505, 'footwear': 0.02572130958425822, 'occasions': 0.06138949907256022, 'seasons': 0.020294544516155647}
month {'accessories': 0.025183709046780585, 'adjectives': 0.09662669185073046, 'clothing': 0.05215095515043095, 'footwear': 0.027867384434255793, 'occasions':

In [27]:
pprint(results)

{'accessories': ['earrings'],
 'adjectives': ['pair'],
 'clothing': ['outfit'],
 'occasions': ['wedding'],
 'seasons': ['month']}


In [28]:
queries = []
cats = ["clothing", "accessories", "footwear"]

for cat in cats:
    if cat in results:
        for item in results[cat]:
            queries.append(item)

print(queries)


['outfit', 'earrings']


In [29]:

occasions = ""
if "occasions" in results:
    for item in results["occasions"]:
        occasions += "+" + item

seasons = ""
if "seasons" in results:
    for item in results["seasons"]:
        seasons += "+" + item

for i in range(len(queries)):
    queries[i] += occasions + "+female"

print(queries)

['outfit+wedding+female', 'earrings+wedding+female']


In [30]:
response = requests.get(f"https://flipkart-scraper-api.dvishal485.workers.dev/search/{queries[0]}").json()

ConnectionError: HTTPSConnectionPool(host='flipkart-scraper-api.dvishal485.workers.dev', port=443): Max retries exceeded with url: /search/outfit+wedding+female (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000136B449A2B0>: Failed to resolve 'flipkart-scraper-api.dvishal485.workers.dev' ([Errno 11001] getaddrinfo failed)"))

In [20]:
total_search_results = response["total_result"]
search_products = response["result"]
search_products

[{'name': 'Latest Collection Kundan Jhumka earrings for Girls and ...',
  'link': 'https://www.flipkart.com/ankev-latest-collection-kundan-jhumka-earrings-girls-woman-jhumki-earring-brass/p/itmc85ec48cfa761',
  'current_price': 284,
  'original_price': 499,
  'discounted': True,
  'thumbnail': 'https://rukminim2.flixcart.com/image/612/612/xif0q/earring/v/n/n/na-2293-queen-goldenborder-ankev-original-imagsyzfy7aajba3.jpeg?q=70',
  'query_url': 'https://flipkart-scraper-api.dvishal485.workers.dev/product/ankev-latest-collection-kundan-jhumka-earrings-girls-woman-jhumki-earring-brass/p/itmc85ec48cfa761'},
 {'name': 'Afghani Oxidised German Silver Tribal Earring for Girls...',
  'link': 'https://www.flipkart.com/fashion-fusion-afghani-oxidised-german-silver-tribal-earring-girls-women-alloy-set/p/itm50a2234c90df6',
  'current_price': 234,
  'original_price': 1599,
  'discounted': True,
  'thumbnail': 'https://rukminim2.flixcart.com/image/612/612/xif0q/earring/q/8/d/na-ffc808-fashion-fusion-

In [11]:
users = pd.read_pickle('users.pkl')
users.head()

,UserID,FirstName,LastName,Age,Gender,Country,ProductsBought,ProductsViewedInLast30Days,ProductsInWishlist
0,352449,Teresa,Johnson,52,M,Micronesia,"[[SGLF84PQ95MAP857, Silver Kartz UV Protection...","[[SNDG5FR793G4VY3Q, Wildcraft Men Multicolor S...","[[SHOGKYNZ9GXSEVWH, Hitpro BwBoots6 Boots For ..."
1,199167,Patrick,Sheppard,35,M,Equatorial Guinea,"[[TOPGG9ZCXMZ4BFFQ, KASSUALLY Casual Sleeveles...","[[TOPGG9ZCXMZ4BFFQ, KASSUALLY Casual Sleeveles...","[[TOGFYHFZDSZJJXMD, ABYS Sling Bag || Backpack..."
2,273321,Michelle,Shields,19,F,El Salvador,"[[DREFECH2HZX4YFKN, MUDRIKA Women Maxi Light B...","[[SARG3GJGXRN2ZSVZ, Flip The Style Printed Bol...","[[SGLGEERZCJKACGMA, METRONAUT UV Protection Su..."
3,230367,David,Powell,32,M,Djibouti,"[[SHOGHGG3M9XTAHYE, MACTREE Classy Trendy Fit-...","[[SGLGQ8VYM22C2FQ7, Hench UV Protection Aviato...","[[SGLGH2GFKPFUDHZG, Intellilens UV Protection,..."
4,383343,Brian,Hamilton,33,M,Tunisia,"[[TOPFAY4BGMKT4T68, KASSUALLY Casual Flared Sl...","[[ERGFYZ6X6NBNJFMD, MEENAZ Traditional South i...","[[SNDFYUYR8VTG3NXE, Paragon Men K1405G Stylish..."


In [31]:
check = SimilarityFinder('sentence-transformers/all-mpnet-base-v2')
heap = []
heapify(heap)
for i in search_products:
    # print("Search Product: ", search_products[i])
    # print("Search Result: ", search_results[i])
    val = 0
    for j in users.iloc[0]["ProductsBought"]:
        val += check.calculate_similarity(
        check.calculate_embeddings(j[1]),
        check.calculate_embeddings(i["name"])).item()
    sim = val/len(users.iloc[0]["ProductsBought"])
    heappush(heap, (-sim , i["name"], i["link"]))
print(heap)
    

[(-0.3685208931565285, 'long chain jhumki earrings double jhumka 2 layer bali e...', 'https://www.flipkart.com/meenaz-long-chain-jhumki-earrings-double-jhumka-2-layer-bali-ear-rings-jhumkas-layered-brass-stone-metal-alloy-copper-clip-on-earring-drops-danglers-earring-set-hoop/p/itma523f3b37a885'), (-0.3538641169667244, 'hoop earrings for women double jhumka 2 layer bali ear ...', 'https://www.flipkart.com/meenaz-hoop-earrings-women-double-jhumka-2-layer-bali-ear-rings-jhumkas-long-chain-brass-stone-metal-alloy-copper-clip-on-earring-drops-danglers-earring-set-jhumki/p/itm560bc5a66ff8c'), (-0.3180521324276924, 'Womens Golden Earcuff Pack of 5 Metal Cuff Earring', 'https://www.flipkart.com/sorellaz-womens-golden-earcuff-pack-5-metal-cuff-earring/p/itm5e4ffbc7451a4'), (-0.3507901281118393, 'Latest Collection Kundan Jhumka earrings for Girls and ...', 'https://www.flipkart.com/ankev-latest-collection-kundan-jhumka-earrings-girls-woman-jhumki-earring-brass/p/itmc85ec48cfa761'), (-0.28285219

In [25]:
sorted_tuples = []
max_heap = heap
while max_heap:
    max_value_neg, value, link = heappop(max_heap)
    sorted_tuples.append((abs(max_value_neg), value, link))

for item in sorted_tuples:
    print(item)

(0.6791191101074219, 'Matte Gold Plated Wedding Jewellery Sets Jhumka/Jhumki ...', 'https://www.flipkart.com/meenaz-matte-gold-plated-wedding-jewellery-sets-jhumka-jhumki-earrings-girls-women-jhumki-earrings-322-beads-pearl-copper-brass-earring/p/itma1b562720cb4e')
(0.6665627360343933, "Women's Gold Plated Jhumka Earrings For Women &amp; Girls|W...", 'https://www.flipkart.com/yuvikrissh-women-s-gold-plated-jhumka-earrings-women-girls-women-s-earring-set-pearl-alloy-jhumki/p/itm72616ea6422a6')
(0.6611420512199402, 'traditional gold plated trendy bali Brass Jhumki Earrin...', 'https://www.flipkart.com/arch-fashion-traditional-gold-plated-trendy-bali-brass-jhumki-earring/p/itm61aaba37b38b5')
(0.6599269509315491, 'Traditional Pearl Jewellery Gold Jhumki Party Wear Styl...', 'https://www.flipkart.com/meenaz-traditional-pearl-jewellery-gold-jhumki-party-wear-stylish-jhumka-earrings-women-girls-necklace-set-earrings-247-cubic-zirconia-copper-brass-earring/p/itm1e74bdfec6325')
(0.6592398285865

In [26]:
print(sorted_tuples[0])

(0.6791191101074219, 'Matte Gold Plated Wedding Jewellery Sets Jhumka/Jhumki ...', 'https://www.flipkart.com/meenaz-matte-gold-plated-wedding-jewellery-sets-jhumka-jhumki-earrings-girls-women-jhumki-earrings-322-beads-pearl-copper-brass-earring/p/itma1b562720cb4e')


In [32]:
print(len(sorted_tuples))

399
